<div style="text-align: right;"><h4>ФБ-21 Редько-Шпак Родислав</h4></div>
<h1><center><b>Лабораторна робота №2</b></center></h1>
<h3><center>З предмета "Засоби підготовки та аналізу даних"</center></h3>
<h5><b><center>Хід роботи:</center></b></h5>

In [32]:
import os
import datetime
import pandas as pd
import urllib.request

print("Setup Complete")

Setup Complete


<p>1. Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу.</p>
<p>2. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області.</p>
<p>3. При зберіганні файлу до його імені потрібно додати дату та час завантаження;</p>

In [33]:
def download_data(province_ID, start_year=2000, end_year=2024):
    # перевіряю, чи існує папка для зберігання
    if not os.path.exists("lab2_VHI"):
        os.makedirs("lab2_VHI")
    
    # перевіряю, чи файл уже завантажений
    filename_pattern = f"VHI-ID_{province_ID}_"
    existing_files = [file for file in os.listdir("lab2_VHI") if file.startswith(filename_pattern)]
    if existing_files:
        print(f"=] Файл для VHI-ID №{province_ID} вже існує: {existing_files[0]}\n")
        return
    
    # скачую
    url_download = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_ID}&year1={start_year}&year2={end_year}&type=Mean"
    vhi_url_open = urllib.request.urlopen(url_download)
    
    # створюю назви файлу з датою та часом
    year_month_now = datetime.datetime.now().strftime("%d-%m-%Y")
    h_m_s_time_now = datetime.datetime.now().strftime("%H-%M-%S")
    filename = f"VHI-ID_{province_ID}_{year_month_now}_{h_m_s_time_now}.csv"
    
    file_path = os.path.join("lab2_VHI", filename)
    with open(file_path, 'wb') as output:
        output.write(vhi_url_open.read())

    # виводжу відповідний текст
    print()
    print(f"=] VHI-файл {filename} завантажений успішно!")
    print(f"=] Тека зберігання: {os.path.abspath(file_path)}")
    print()

    return

# приклад
for index in range(1, 28):
    print(f"!] Завантаження CSV-файлу за VHI-ID №{index}...")
    download_data(index, 2000, 2024)


!] Завантаження CSV-файлу за VHI-ID №1...

=] VHI-файл VHI-ID_1_08-03-2024_12-11-10.csv завантажений успішно!
=] Тека зберігання: c:\Users\rdk\Documents\ad_lab2\lab2_VHI\VHI-ID_1_08-03-2024_12-11-10.csv

!] Завантаження CSV-файлу за VHI-ID №2...

=] VHI-файл VHI-ID_2_08-03-2024_12-11-10.csv завантажений успішно!
=] Тека зберігання: c:\Users\rdk\Documents\ad_lab2\lab2_VHI\VHI-ID_2_08-03-2024_12-11-10.csv

!] Завантаження CSV-файлу за VHI-ID №3...

=] VHI-файл VHI-ID_3_08-03-2024_12-11-12.csv завантажений успішно!
=] Тека зберігання: c:\Users\rdk\Documents\ad_lab2\lab2_VHI\VHI-ID_3_08-03-2024_12-11-12.csv

!] Завантаження CSV-файлу за VHI-ID №4...

=] VHI-файл VHI-ID_4_08-03-2024_12-11-13.csv завантажений успішно!
=] Тека зберігання: c:\Users\rdk\Documents\ad_lab2\lab2_VHI\VHI-ID_4_08-03-2024_12-11-13.csv

!] Завантаження CSV-файлу за VHI-ID №5...

=] VHI-файл VHI-ID_5_08-03-2024_12-11-14.csv завантажений успішно!
=] Тека зберігання: c:\Users\rdk\Documents\ad_lab2\lab2_VHI\VHI-ID_5_08-03

1) Зчитати завантажені текстові файли у фрейм. 
2) Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). 
3) Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [34]:
def dataframer(folder_path):
    fr, columns  = [], ["Year", "Week", "SMN", "SMT", "VCI", "TCI", "VHI", "empty"]

    # отримую список файлів CSV у папці
    csv_files = filter(lambda x: x.endswith('.csv'), os.listdir(folder_path))

    # перебираю файли
    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        province_ID = int(file_name.split('_')[1])
        
        # зчитую та обробляю CSV файли
        df = pd.read_csv(file_path, header=1, names=columns) # зчитую файл CSV у df
        df.at[0, "Year"] = df.at[0, "Year"][9:] # виправляю рік у стовпці Year

        df = df.drop(df.index[-1]) # видаляю останній рядокі
        df = df.drop(df.loc[df["VHI"] == -1].index) # видаляю рядки з невизначеним VHI
        df = df.drop("empty", axis=1) # видаляю порожній стовбець

        df.insert(0, "province_ID", province_ID, True) # вставяю стовбець 'province_ID'
        df['Year'] = df['Year'].astype(int) # Year у ціле число.
        df["Week"] = df["Week"].astype(int) # Week у ціле число.

        fr.append(df) # додаю до списку

    # об'єдную дані у фрейм, видаливши дублікати
    df_res = pd.concat(fr).drop_duplicates().reset_index(drop=True)
    # відкидаю області 12 та 20
    df_res = df_res.loc[(df_res.province_ID != 12) & (df_res.province_ID != 20)]

    return df_res

# ф-ія для гарного виводу тексту
def format_result(df):
    result_string = ""
    for province_ID, province_data in df.groupby('province_ID'):
        # для кожної області обираю приклад даних початкового та останнього року
        start_year_example_data = province_data[province_data['Year'] == province_data['Year'].min()].head(1)
        end_year_example_data = province_data[province_data['Year'] == province_data['Year'].max()].tail(1)
        
        result_string += "\n\n\n" + " "*17 + f"[= Province ID: {province_ID} =]\n"
        result_string += "=" * 55 + "\n"
        
        
        result_string += " "*12 + "Приклад даних початкового року:\n"
        result_string += "-" * 55 + "\n"
        result_string += f"province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI\n"
        result_string += "-" * 55 + "\n"
        result_string += start_year_example_data.to_string(index=False, header=False).replace('\n', f'\n{province_ID}| ').replace(' ', ' | ') + "\n"
        result_string += " " * 26 + "..." + " " * 26 + "\n"
        
        result_string += " "*12 + "Приклад даних останнього року:\n"
        result_string += "-" * 55 + "\n"
        result_string += f"province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI\n"
        result_string += "-" * 55 + "\n"
        result_string += end_year_example_data.to_string(index=False, header=False).replace('\n', f'\n{province_ID}| ').replace(' ', ' | ') + "\n"
        
    return result_string

# приклад
folder_path = "lab2_VHI"
data_frame = dataframer(folder_path)
result_string = format_result(data_frame)
print(result_string)




                 [= Province ID: 1 =]
            Приклад даних початкового року:
-------------------------------------------------------
province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI
-------------------------------------------------------
1 | 2000 | 1 | 0.031 | 257.0 | 20.04 | 51.54 | 35.79
                          ...                          
            Приклад даних останнього року:
-------------------------------------------------------
province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI
-------------------------------------------------------
1 | 2024 | 9 | 0.111 | 272.32 | 61.95 | 30.3 | 46.07



                 [= Province ID: 2 =]
            Приклад даних початкового року:
-------------------------------------------------------
province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI
-------------------------------------------------------
2 | 2000 | 1 | 0.044 | 262.34 | 18.62 | 31.06 | 24.84
                          ...                          
           

<p><b> Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на наступні: </b></p>
<p> | № області | Назва | </p>
<p> ----------------------- </p>
<p> |....................... | ............. | </p>

In [35]:
def change_province_id(df):
    # словник між старими і новими індексами областей
    province_mapping = {
        1: 22,
        2: 24,
        3: 23,
        4: 25,
        5: 3,
        6: 4,
        7: 8,
        8: 19,
        9: 20,
        10: 21,
        11: 9,
        13: 10,
        14: 11,
        15: 12,
        16: 13,
        17: 14,
        18: 15,
        19: 16,
        21: 17,
        22: 18,
        23: 6,
        24: 1,
        25: 2,
        26: 7,
        27: 5,
    }
    
    # міняю індекси областей у фреймі
    df_copy = df.copy() # попередньо зробивши копію фрейма міняю
    df_copy['province_ID'] = df_copy['province_ID'].replace(province_mapping)
    
    return format_result(df_copy)

# приклад
folder_path = "lab2_VHI"
data_frame = dataframer(folder_path)
changed_id_result = change_province_id(data_frame)
print(changed_id_result)





                 [= Province ID: 1 =]
            Приклад даних початкового року:
-------------------------------------------------------
province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI
-------------------------------------------------------
1 | 2000 | 1 | 0.023 | 260.25 | 8.46 | 39.97 | 24.22
                          ...                          
            Приклад даних останнього року:
-------------------------------------------------------
province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI
-------------------------------------------------------
1 | 2024 | 9 | 0.117 | 274.51 | 65.89 | 26.44 | 46.16



                 [= Province ID: 2 =]
            Приклад даних початкового року:
-------------------------------------------------------
province_ID | Year | Week | SMN | SMT | VCI | TCI | VHI
-------------------------------------------------------
2 | 2000 | 1 | 0.041 | 264.68 | 13.12 | 36.19 | 24.65
                          ...                          
          

<p><b> Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу): </b></p>
<p>1. Ряд VHI для області за рік, пошук екстремумів (min та max);</p>

In [36]:
def calculate_vhi_extremes(df, start_year=None, end_year=None):

    # якщо рік не заданий - ставлю початковий/останній рік датафрейму
    if start_year is not None:
        df = df[df['Year'] >= start_year]
    if end_year is not None:
        df = df[df['Year'] <= end_year]

    # групую дані за областями та роками
    grouped_data = df.groupby(['province_ID', 'Year'])

    # список для результату
    vhi_extremes = []

    # перебираю кожну групу
    for (province_id, year), group_df in grouped_data:
        # знаходжу min та max значення VHI для кожної області та року
        min_vhi = group_df['VHI'].min()
        max_vhi = group_df['VHI'].max()

        # додаю результат до списку
        vhi_extremes.append({'province_ID': province_id, 'Year': year, 'Min_VHI': min_vhi, 'Max_VHI': max_vhi})

    # df з результатами
    result_df = pd.DataFrame(vhi_extremes)

    return result_df

# приклад
vhi_extremes_df = calculate_vhi_extremes(data_frame, start_year=2005, end_year=2024)
print(vhi_extremes_df)


     province_ID  Year  Min_VHI  Max_VHI
0              1  2005    41.62    68.89
1              1  2006    35.84    66.33
2              1  2007    26.46    52.82
3              1  2008    25.87    74.45
4              1  2009    36.76    62.66
..           ...   ...      ...      ...
495           27  2020    35.05    58.33
496           27  2021    36.11    65.33
497           27  2022    30.36    63.81
498           27  2023    32.92    64.08
499           27  2024    42.30    50.45

[500 rows x 4 columns]


<p><b> Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу): </b></p>
<p> 2. Ряд VHI за всі роки для області, виявити роки з екстремальними посухами, які торкнулися більше вказаного відсотка областей; </p>
<p> 3. Аналогічно для помірних посух </p>

In [38]:
def drought_years(df, percentage, type_of_drought, start_year=None, end_year=None):
    # якщо рік не заданий - ставлю початковий/останній рік датафрейму
    if start_year is not None:
        df = df[df['Year'] >= start_year]
    if end_year is not None:
        df = df[df['Year'] <= end_year]

    # словник кількості областей з VHI менше порогового значення для кожного року
    drought_count = {}

    # групую дані за роками
    grouped_data = df.groupby('Year')

    # перебираю кожний рік
    for year, group_df in grouped_data:
        # знаходжу к-сть областей у цьому році
        total_areas = len(group_df['province_ID'].unique())

        if type_of_drought == "extreme":
            # к-сть областей з VHI < 15
            drought_areas = group_df[group_df['VHI'] < 15]['province_ID'].nunique()
        elif type_of_drought == "moderate":
            # к-сть областей з VHI > 15 але < 35
            drought_areas = group_df[(group_df['VHI'] > 15) & (group_df['VHI'] < 35)]['province_ID'].nunique()
        
        # якщо к-сть посушливих областей > заданий відсоток - то зберігаю рік
        if drought_areas / total_areas > percentage / 100:
            drought_count[year] = (drought_areas, round((drought_areas / total_areas) * 100, 2))

    # виводжу у вигляді таблиці
    if type_of_drought == "extreme":
        result_table = f"Роки з екстремальною посухою (де більше {percentage}% областей мають VHI < 15):\n"

    elif type_of_drought == "moderate":
        result_table = f"Роки з помірною посухою (де більше {percentage}% областей мають VHI від 15 до 35):\n"

    result_table += "-" * 60 + "\n"
    result_table += f"{'Year':^10} | {'К-сть областей з помірною посухою':^25} | {'Відсоток':^15}\n"
    result_table += "-" * 60 + "\n"

    for year, (drought_areas, percent) in drought_count.items():
        result_table += f"{year:^10} | {drought_areas:^25} | {percent:^15}%\n"

    result_table += "-" * 60 + "\n"

    return result_table

# приклад для екстримальних
drought_years_extreme = drought_years(data_frame, 10, "extreme", start_year=2000, end_year=2024,)
print(drought_years_extreme)

# приклад для помірних
drought_years_moderate = drought_years(data_frame, 10, "moderate", start_year=2000, end_year=2024,)
print(drought_years_moderate)


Роки з екстремальною посухою (де більше 10% областей мають VHI < 15):
------------------------------------------------------------
   Year    | К-сть областей з помірною посухою |    Відсоток    
------------------------------------------------------------
   2000    |             4             |      16.0      %
   2007    |             5             |      20.0      %
------------------------------------------------------------

Роки з помірною посухою (де більше 10% областей мають VHI від 15 до 35):
------------------------------------------------------------
   Year    | К-сть областей з помірною посухою |    Відсоток    
------------------------------------------------------------
   2000    |            25             |      100.0     %
   2001    |            10             |      40.0      %
   2002    |            12             |      48.0      %
   2003    |            13             |      52.0      %
   2004    |             3             |      12.0      %
   2005    |   